# **Sesión 3:** Procesamiento de imagenes y extracción de características

## **Librerías**

In [1]:
import cv2
print("OpenCV should be 4.8.0.76 Current version:", cv2.__version__)
from typing import List
import numpy as np
import imageio
import copy
import glob
import matplotlib.pyplot as plt
from typing import Optional
from utils import *
import os

OpenCV should be 4.8.0.76 Current version: 4.8.0


## **Apartado A:** Filtro Gaussiano y Detección de bordes: Sobel y Canny

El objetivo de este apartado es detectar los bordes de las imágenes de la carpeta ``data/partA-B``. Para ello, deberá seguir los siguientes pasos:

1. **Tarea A.1.** Defina el método ``gaussian_blur()`` que aplique un filtro gausiano para obtener imágenes borrosas. Siga todas las indicaciones del enunciado.
2. **Tarea A.2.** Aplique el método ``gaussian_blur()`` a todas las imágenes en ``data/partA-B``.


3. **Tarea A.3.** Defina la función ``sobel_edge_detector()`` que detecte bordes con el método Sobel. Siga todas las indicaciones del enunciado.
4. **Tarea A.4.** Aplique el método ``sobel_edge_detector()`` a todas las imágenes en ``data/partA-B``.


5. **Tarea A.5.** Defina la función ``canny_edge_detector()`` que detecte bordes con el método Canny. Siga todas las indicaciones del enunciado.
6. **Tarea A.6.** Aplique el método ``canny_edge_detector()`` a todas las imágenes en ``data/partA-B``.

### **Tarea A.1:** Defina el método ``gaussian_blur()`` que aplique un filtro gausiano para obtener imágenes borrosas.

In [2]:
def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames]

In [3]:
def show_image(img: np.array, img_name: str = "Image"):
    cv2.imshow(img_name,img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [4]:
def write_image(output_folder: str, img_name: str, img: np.array):
    os.makedirs(output_folder, exist_ok=True)
    img_path = os.path.join(output_folder,img_name)
    cv2.imwrite(img_path,img)

In [5]:
# TODO Define the method
def gaussian_blur(img: np.array, sigma: float, filter_shape: Optional[List] = None, verbose: bool = False) -> np.array:
    # TODO If not given, compute the filter shape 
    if filter_shape == None:
        filter_size = int(2 * np.ceil(3 * sigma) + 1) 
        filter_l = [filter_size, filter_size]

    else:
        filter_l = filter_shape
    
    # TODO Create the filter coordinates matrices
    y, x = np.mgrid[-filter_l[0]//2 + 1:filter_l[0]//2 + 1,
                    -filter_l[1]//2 + 1:filter_l[1]//2 + 1]

    
    # TODO Define the formula that goberns the filter
    formula = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    gaussian_filter = formula / formula.sum()
    
    # TODO Process the image
    gb_img = cv2.filter2D(img, -1, gaussian_filter)
    
    if verbose:
        show_image(img=gb_img, img_name=f"Gaussian Blur: Sigma = {sigma}")
    
    return gaussian_filter, gb_img.astype(np.uint8)

### **Tarea A.2.** Aplique el método ``gaussian_blur()`` a todas las imágenes en ``data``.

In [6]:
# TODO Get the gaussian blurred images using a list comprehension
filenames = [f"../data/partA-B/{i:03d}.jpg" for i in range(6)]
imgs = load_images(filenames)

gauss_sigma = 1.5
gb_imgs = [gaussian_blur(img,sigma=gauss_sigma,verbose=True)[1] for img in imgs]


for i,gb in zip(range(len(gb_imgs)),gb_imgs):
    write_image(f"../OutputA-B",f"Gauss_{i:03d}.jpg",gb)

gb_imgs = [gaussian_blur(img,sigma=gauss_sigma,verbose=False) for img in imgs]

C:\Users\rayne\AppData\Local\Temp\ipykernel_26432\283608422.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


### **Tarea A.3:** Defina la función ``sobel_edge_detector()`` que detecte bordes con el método Sobel.

In [7]:
# TODO Define the method


def sobel_edge_detector(img: np.array, filter: np.array, gauss_sigma: float, gauss_filter_shape: Optional[List] = None, verbose: bool = False) -> np.array:
    # TODO Transform the img to grayscale
    if len(img.shape) == 3 and img.shape[2] == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray_img = img.copy()

    
    # TODO Get a blurry img to improve edge detections
    blurred = gaussian_blur(img=gray_img, sigma=gauss_sigma, filter_shape=gauss_filter_shape, verbose=verbose)[1]
    
    # Re-scale
    blurred = blurred/255
    
    # TODO Get vertical edges
    v_edges = cv2.filter2D(blurred, -1, filter)
    
    # TODO Transform the filter to get the orthogonal edges
    h_filter = filter.T
    
    # TODO Get horizontal edges
    h_edges = cv2.filter2D(blurred, -1, h_filter)
    
    # TODO Get edges
    sobel_edges_img = np.hypot(h_edges, v_edges)
   

    
    # Get edges angle
    theta = np.arctan2(v_edges, h_edges)

    
    # Visualize if needed
    if verbose:
        show_image(img=sobel_edges_img, img_name="Sobel Edges")
    
    return np.squeeze(sobel_edges_img), np.squeeze(theta)

### **Tarea A.4.** Aplique el método ``sobel_edge_detector()`` a todas las imágenes en ``data``.

In [8]:
# TODO Define a sigma value
gauss_sigma = 1.5

# TODO Define the Sobel filter
sobel_filter = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])


# TODO Get the edges detected by Sobel using a list comprehension
sobel_edges_imgs = [sobel_edge_detector(img,sobel_filter,gauss_sigma,verbose=True) for img in imgs]

for i,sobel in zip(range(len(sobel_edges_imgs)),sobel_edges_imgs):
    sobel_img = sobel[0]/ sobel[0].max() * 255
    write_image(f"../OutputA-B",f"Sobel_edges_{i:03d}.jpg",sobel_img)




### **Tarea A.5:** Defina la función ``canny_edge_detector()`` que detecte bordes con el método Canny.

In [9]:
# TODO Define the method
def canny_edge_detector(img: np.array, sobel_filter: np.array, gauss_sigma: float, gauss_filter_shape: Optional[List] = None, verbose: bool = False):
    # TODO Call the method sobel_edge_detector()
    sobel_edges_img, theta = sobel_edge_detector(img,sobel_filter,gauss_sigma,gauss_filter_shape,verbose)
    
    # TODO Use NMS to refine edges
    canny_edges_img = non_max_suppression(sobel_edges_img,theta)
    
    if verbose:
        show_image(canny_edges_img, img_name="Canny Edges")
        
    return canny_edges_img

### **Tarea A.6.** Aplique el método ``canny_edge_detector()`` a todas las imágenes en ``data``.

In [10]:
# TODO Define Sobel filter
sobel_filter = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])

# TODO Define a sigma value for Gauss
gauss_sigma = 1.5

# TODO Define a Gauss filter shape
gauss_filter_shape = [5, 5]

# TODO Get the edges detected by Canny using a list comprehension
canny_edges = [canny_edge_detector(img,sobel_filter,gauss_sigma,gauss_filter_shape,True) for img in imgs]
for i,canny in zip(range(len(canny_edges)),canny_edges):
    canny_img = canny/ canny.max() * 255
    write_image(f"../OutputA-B",f"Canny_edges_{i:03d}.jpg",canny_img)

### **Pregunta A.1:** Añada ruido a las imágenes de la carpeta ``data``. Compare los resultados que obtiene al aplicar su filtro Sobel con y sin filtro Gausiano.

In [11]:
# TODO Homework

def add_gaussian_noise(img: np.array, mean: float = 0, stddev: float = 25) -> np.array:
    noise = np.random.normal(mean, stddev, img.shape).astype(np.float32)
    noisy_img = img.astype(np.float32) + noise
    noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)
    return noisy_img

noisy_imgs = [add_gaussian_noise(img) for img in imgs]

sobel_filtered = [sobel_edge_detector(img,sobel_filter,gauss_sigma,verbose=True) for img in noisy_imgs]
sobel_unfiltered = [sobel_edge_detector(img,sobel_filter,1,[1,1],verbose=True) for img in noisy_imgs]

for i,filtered,unfiltered in zip(range(len(canny_edges)),sobel_filtered,sobel_unfiltered):
    filtered_img = filtered[0]/ filtered[0].max() * 255
    unfiltered_img = unfiltered[0]/ unfiltered[0].max() * 255
    write_image(f"../OutputA-B",f"filtered_Sobel_{i:03d}.jpg",filtered_img)
    write_image(f"../OutputA-B",f"unfiltered_Sobel_{i:03d}.jpg",unfiltered_img)

#En las que no hay filtro se detectan más bordes que en las que si se aplicó

### **Pregunta A.2:** Utilice la librería ``scikit-image`` y compare el efecto de los filtros Sobel, Canny y Prewitt sobre las imágenes de la carpeta ``data``. ¿Qué diferencias observa entre los filtros? ¿Puede obtener alguna conclusión y/o patrón?

In [18]:
# TODO Homework
from skimage import filters, color, feature

gray_imgs = [color.rgb2gray(img) for img in imgs ]
sobel_edges =[filters.sobel(gray_img) for gray_img in gray_imgs]
prewitt_edges = [filters.prewitt(gray_img) for gray_img in gray_imgs]
canny_edges = [feature.canny(gray_img, sigma=1.5)/feature.canny(gray_img, sigma=1.5).max()*255 for gray_img in gray_imgs]

for sobel,prewitt,canny,i in zip(sobel_edges,prewitt_edges,canny_edges,range(len(gray_imgs))):
    show_image(sobel, img_name="Sobel Edges")
    show_image(prewitt, img_name="Prewitt Edges")
    show_image(canny, img_name="Canny Edges")
    sobel = sobel/sobel.max()*255
    prewitt = prewitt/prewitt.max()*255
    write_image(f"../OutputA-B",f"scikit_Sobel_{i:03d}.jpg",sobel)
    write_image(f"../OutputA-B",f"scikit_Prewitt_{i:03d}.jpg",prewitt)
    write_image(f"../OutputA-B",f"scikit_Canny_{i:03d}.jpg",canny)
  

    
#- Canny es el más robusto frente al ruido y produce bordes más coherentes, especialmente en imágenes con textura o detalles finos.
#- Sobel y Prewitt son útiles para análisis rápido de gradientes, pero requieren suavizado previo para evitar falsos positivos.
#- En general, Canny es preferible para tareas de segmentación o análisis estructural, mientras que Sobel y Prewitt pueden ser útiles como pasos intermedios o para visualización de gradientes.


## **Apartado B:** Operadores Morfológicos

Para resolver este partado, deberá seguir los siguientes pasos:

1. **Tarea B.1.** Defina el método ``binarize()`` para binarizar imágenes.
2. **Tarea B.2.** Defina el método ``custom_dilate()``.
3. **Tarea B.3.** Defina el método ``custom_erode()``.
4. **Pregunta B.1** Aplique los métodos ``custom_dilate()`` y ``custom_erode()`` a todas las imágenes de la carpeta ``data``.


### **Tarea B.1.** Defina el método ``binarize()`` para binarizar imágenes.

In [13]:
# TODO Homework: define the binarization method
def binarize(img: np.array, threshold: int = 127):
    binary_img = (img > threshold).astype(np.uint8) 

    return binary_img

### **Tarea B.2.** Defina el método ``custom_dilate()``

In [14]:
# TODO Homework: define the dilation method
def custom_dilate(img):
    # TODO pad the original image so it can keep dimensions after processing
    padded = np.pad(img, pad_width=1, mode='constant', constant_values=0)

    
    # TODO get img shape
    width = img.shape[1]
    height = img.shape[0]
   

    # TODO Create an element with the same dimensions as the padded img
    dilated = np.zeros_like(padded)

    
    for j in range(1, height + 1):
        for i in range(1, width + 1):
            region = padded[j-1:j+2, i-1:i+2]
            if np.any(region == 1):
                dilated[j, i] = 1

            
    # TODO Select the region of interest (ROI). Modify if needed
    dilated = dilated[1:height+1, 1:width+1]
    
    return dilated

### **Tarea B.3.** Defina el método ``custom_erode()``

In [15]:
# TODO Homework: define the erotion method
def custom_erode(img):
    padded = np.pad(img, pad_width=1, mode='constant', constant_values=0)
    width = img.shape[1]
    height = img.shape[0]
    eroded = np.zeros_like(padded)

    for j in range(1, height + 1):
            for i in range(1, width + 1):
                region = padded[j-1:j+2, i-1:i+2]
                if np.all(region == 1):
                    eroded[j, i] = 1

    eroded = eroded[1:height+1, 1:width+1]
    
    return eroded

### **Pregunta B.1** Aplique los métodos ``custom_dilate()`` y ``custom_erode()`` a todas las imágenes de la carpeta ``data``.

In [16]:
# TODO Homework
binary_imgs = [binarize(gray_img/gray_img.max()*255) for gray_img in gray_imgs]
dilated_imgs =[custom_dilate(b)*255 for b in binary_imgs]
eroded_imgs = [custom_erode(b)*255 for b in binary_imgs]



for dilated,eroded,i in zip(dilated_imgs,eroded_imgs,range(len(gray_imgs))):
    show_image(dilated, img_name="dilated")
    show_image(eroded, img_name="eroded")
    write_image(f"../OutputA-B",f"dilated_{i:03d}.jpg",dilated)
    write_image(f"../OutputA-B",f"eroded_{i:03d}.jpg",eroded)
 